In [1]:
import csv
import pandas as pd
import numpy as np
from random import randint
import time
from tqdm import tqdm, tqdm_notebook
import math
import pickle
print('Done')

Done


In [2]:
tweets=[]
start= time.time()
train_data= pd.read_csv('data/training.csv',names = ['Type' , 'id', 'date', 'query','user', 'tweet'], encoding='latin-1')
test_data= pd.read_csv('data/test.csv',names = ['Type' , 'id', 'date', 'query','user', 'tweet'], encoding='latin-1')
test_data = test_data.drop(['id','date','query','user'], axis=1)
train_data= train_data.drop(['id','date','query','user'], axis=1)
test_data = test_data.drop(test_data[test_data['Type']==2].index)
# print(train_data[:5])
# print(train_data[:5])
print('time',time.time()-start)
print(train_data.shape)
print(test_data.shape)

time 4.813804388046265
(1600000, 2)
(359, 2)


In [3]:
tweets = train_data.set_index('Type')
tweets['words']= tweets['tweet'].str.replace(',',' ').str.replace('.', ' ').str.split()
tweets['word_count']= tweets['words'].str.len()
tweets = tweets.drop(['tweet'], axis=1)

# tweets_test = test_data.set_index('Type')
tweets_test=test_data
tweets_test['words']= tweets_test['tweet'].str.replace(',',' ').str.replace('.', ' ').str.split()
tweets_test['word_count']= tweets_test ['words'].str.len()
tweets_test = tweets_test.set_index('Type')
tweets_test = tweets_test.drop(['tweet'], axis=1)
print(tweets.shape)
print(tweets_test.shape)
print(tweets[:5])

(1600000, 2)
(359, 2)
                                                  words  word_count
Type                                                               
0     [@switchfoot, http://twitpic, com/2y1zl, -, Aw...          20
0     [is, upset, that, he, can't, update, his, Face...          21
0     [@Kenichan, I, dived, many, times, for, the, b...          18
0     [my, whole, body, feels, itchy, and, like, its...          10
0     [@nationwideclass, no, it's, not, behaving, at...          21


In [4]:
def create_vocab(tweets):
    vocab = {}
    for l_words in tweets['words']:
        for word in l_words:
            vocab[word] =1
    return vocab

In [5]:
def create_dict(tweets,vocab):
    phi_neg = (tweets_neg.shape[0]+1)/(tweets.shape[0] +2)
    phi_pos =(tweets_pos.shape[0]+1)/(tweets.shape[0]+2)
        
    dict_neg_words={}
    dict_neg_words=vocab.copy()
    
    for l_words in tweets_neg['words']:
        for word in l_words:
                dict_neg_words[word] +=1
    
    dict_pos_words={}
    dict_pos_words=vocab.copy()
    for l_words in tweets_pos['words']:
        for word in l_words:
            dict_pos_words[word] +=1
    return dict_neg_words, dict_pos_words, phi_neg, phi_pos

In [6]:
print(12040935-1078212)
11423286-1078212

10962723


10345074

In [58]:
def find_accuracy(y, y_pred):
    correct =0
    for i in range(len(y)):
        if y[i] == y_pred[i]:
            correct +=1
    return correct/len(y)

In [7]:
def naive_bayes():   
    count=0
    correct =0
    total_words_neg = tweets_neg['word_count'].sum()
    total_words_pos = tweets_pos['word_count'].sum()
    total_words_vocab = len(vocab)
    print(total_words_neg)
    print(total_words_vocab)
    for word in vocab:
        theta_neg[word]= dict_neg_words[word] /(total_words_neg+total_words_vocab)
        theta_pos[word]= dict_pos_words[word] /(total_words_pos + total_words_vocab)   

In [8]:
def check_accuracy(tweets):
    tweets_neg=tweets.loc[[0]]
    tweets_pos=tweets.loc[[4]]
    total_words_neg = tweets_neg['word_count'].sum()
    total_words_pos = tweets_pos['word_count'].sum()
    total_words_vocab = len(vocab)
    temp_n = math.log(1/(total_words_neg+total_words_vocab))
    temp_p = math.log(1/(total_words_pos + total_words_vocab))
    correct =0
    for tweet in tweets_neg['words']:
        prob_y_neg =0
        prob_y_pos =0
        for word in tweet:
            if word in vocab:
                prob_y_neg += math.log(theta_neg[word])
                prob_y_pos += math.log(theta_pos[word])
            else :
                prob_y_pos += temp_p
                prob_y_neg += temp_n
        prob_y_neg += math.log(phi_neg)
        prob_y_pos += math.log(phi_pos)
        if(prob_y_neg > prob_y_pos):
            correct += 1
    print(correct) 
    correct_neg = correct
        
    for tweet in tweets_pos['words']:
        prob_y_neg =0
        prob_y_pos =0
        for word in tweet:
            if word in theta_neg:
                prob_y_neg += math.log(theta_neg[word])
                prob_y_pos += math.log(theta_pos[word])
            else :
                prob_y_pos += temp_p
                prob_y_neg += temp_n
        prob_y_neg += math.log(phi_neg)
        prob_y_pos += math.log(phi_pos)
        if(prob_y_neg < prob_y_pos):
            correct += 1   
    correct_pos = correct- correct_neg       
    print(correct_pos)
    print('accuray ',correct/tweets.shape[0])
    return correct_neg, correct_pos

In [9]:
start= time.time()
tweets_neg=tweets.loc[[0]]
tweets_pos=tweets.loc[[4]]
vocab = create_vocab(tweets)
dict_neg_words, dict_pos_words, phi_neg, phi_pos= create_dict(tweets,vocab)
theta_neg={}
theta_pos={}
naive_bayes()
print(time.time()-start)
print('Done')

10962673
1078262
9.789448261260986
Done


In [10]:
# for key in theta_neg:
#     print(key, theta_neg[key])

In [11]:
start = time.time()
check_accuracy(tweets)
print('time', time.time()- start)
print('Done')   

712613
646313
accuray  0.84932875
time 15.832371950149536
Done


In [12]:
total_neg_test = tweets_test.loc[[0]].shape[0]
total_pos_test = tweets_test.loc[[4]].shape[0]
start = time.time()
correct_neg, correct_pos = check_accuracy(tweets_test)
print(time.time()-start)
print(tweets_test[-10:])

142
148
accuray  0.807799442896936
0.006784200668334961
                                                  words  word_count
Type                                                               
0     [Monday, already, Iran, may, implode, Kitchen,...          22
4     [getting, ready, to, test, out, some, burger, ...          20
4     [i, lam, so, in, love, with, Bobby, Flay, he, ...          26
0     [I, just, created, my, first, LaTeX, file, fro...          22
4     [using, Linux, and, loving, it, -, so, much, n...          19
4     [After, using, LaTeX, a, lot, any, other, type...          12
0     [On, that, note, I, hate, Word, I, hate, Pages...          27
4     [Ahhh, back, in, a, *real*, text, editing, env...          11
0     [Trouble, in, Iran, I, see, Hmm, Iran, Iran, s...          12
0     [Reading, the, tweets, coming, out, of, Iran, ...          15


In [13]:
#confusion matrix
wrong_neg = total_neg_test-correct_neg
wrong_pos = total_pos_test-correct_pos
con_mat = pd.DataFrame([[correct_neg, wrong_pos],[wrong_neg, correct_pos]], columns=['neg_actual', 'pos_actual'], index=['neg_pred', 'pos_pred'])

In [14]:
print(con_mat)

          neg_actual  pos_actual
neg_pred         142          34
pos_pred          35         148


## b)  Random Prediction

In [15]:
def random_pred():
    correct =0
    for ind in tweets_test.index:
        temp = randint(0,1)
        if temp ==1:
            temp =4
        if(ind == temp):
            correct += 1
#             print(ind, temp)
    print(correct)
    print('accuracy ', correct/tweets_test.shape[0])    

In [16]:
random_pred()

189
accuracy  0.5264623955431755


### Majority

In [17]:
correct =0
for ind in tweets_test.index:
    maj =4
    if tweets_neg.shape[0] > tweets_pos.shape[0]:
        maj = tweets_neg.shape[0]
    if(maj == ind):
        correct+=1
print(correct/tweets_test.shape[0])

0.5069637883008357


1.(d) stop word removal

In [18]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemming = PorterStemmer()
print('Done')

Done


In [19]:
stop_words = set(stopwords.words('english'))

In [20]:
# t = train_data
# print(t.shape[0])
# start = time.time()
# t['words']=(t['tweet'].str.lower()).map(word_tokenize)
# # tweets['word_count']= tweets['words'].str.len()
# # tweets = tweets.drop(['tweet'], axis=1)
# print(time.time()- start)
# print(t[:12])
# print('Done')
# t = t.drop(['tweet'], axis=1)
# t.to_pickle('df_tweet')

In [21]:
def sep(str):
    temp= str.lower().replace(',',' ').replace('.',' ').split()
    res =[stemming.stem(word) for word in temp if word not in stop_words and '@' not in word]        
    return res        

In [22]:
tweets = train_data.set_index('Type')
start = time.time()
tweets['words']= tweets['tweet'].map(sep)
tweets['word_count']= tweets['words'].str.len()
tweets = tweets.drop(['tweet'], axis=1)
print(time.time()-start)
print('Done')
print(tweets[:5])
print('Done')

231.01471519470215
Done
                                                  words  word_count
Type                                                               
0     [http://twitpic, com/2y1zl, -, awww, that', bu...          13
0     [upset, can't, updat, facebook, text, might, c...          12
0     [dive, mani, time, ball, manag, save, 50%, res...          10
0                [whole, bodi, feel, itchi, like, fire]           6
0                  [behav, i'm, mad, here?, can't, see]           6
Done


In [23]:
start= time.time()
tweets_neg=tweets.loc[[0]]
tweets_pos=tweets.loc[[4]]
vocab = create_vocab(tweets)
dict_neg_words={}
dict_pos_words={}
dict_neg_words, dict_pos_words, phi_neg, phi_pos= create_dict(tweets,vocab)
theta_neg={}
theta_pos={}
naive_bayes()
print(time.time()-start)
print('Done')

6138839
545693
6.948061227798462
Done


In [24]:
start = time.time()
check_accuracy(tweets)
print('time', time.time()- start)
print('Done')        

663939
626842
accuray  0.806738125
time 10.596755027770996
Done


In [25]:
tweets_test = test_data.set_index('Type')
start = time.time()
tweets_test['words']= tweets_test['tweet'].map(sep)
tweets_test['word_count']= tweets_test['words'].str.len()
tweets_test = tweets_test.drop(['tweet'], axis=1)
print(time.time()-start)

0.07632899284362793


In [26]:
start = time.time()
correct_neg, correct_pos = check_accuracy(tweets_test)
print(time.time()-start)
# print(tweets_test)

146
153
accuray  0.8328690807799443
0.01022481918334961


# (e) TF-IDF

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.naive_bayes import GaussianNB

In [28]:
def merge(list):
    return ' '.join(list)

tweets['tweet']= tweets['words'].map(merge)
# print(tweets)

In [29]:
tfidfVectorizer=TfidfVectorizer(use_idf=True)
 
tfidf_vectors=tfidfVectorizer.fit_transform(tweets['tweet'].tolist())

print('Done')

Done


In [30]:
print(tfidf_vectors.shape)
# print(tfidf_vectors)

(1600000, 325212)


In [31]:
# len(train_data['Type'].toarray().tolist())

In [32]:
# nb_model=GaussianNB()
# for i in tqdm(range(0,1600000,1000),total=1600):
#     X_small=tfidf_vectors[i:i+1000,:]
#     nb_model.partial_fit(X_small.todense(),train_data[i:i+1000]['Type'].tolist(),[0,4])

In [36]:
filename = 'finalized_model.sav'
# pickle.dump(nb_model, open(filename, 'wb'))

In [37]:
loaded_model = pickle.load(open(filename, 'rb'))

In [38]:
# pred_y = loaded_model.predict(text)

In [39]:
tweets_test['tweet']= tweets_test['words'].map(merge)
print(tweets_test[:5])



                                                  words  word_count  \
Type                                                                  
4     [loooooooovvvvvvee, kindle2, dx, cool, 2, fant...           7   
4         [read, kindle2, love, lee, child, good, read]           7   
4          [ok, first, asses, #kindle2, fuck, rocks!!!]           6   
4     [love, kindle2, i'v, mine, month, never, look,...          15   
4           [fair, enough, kindle2, think, perfect, :)]           6   

                                                  tweet  
Type                                                     
4     loooooooovvvvvvee kindle2 dx cool 2 fantast right  
4                 read kindle2 love lee child good read  
4                 ok first asses #kindle2 fuck rocks!!!  
4     love kindle2 i'v mine month never look back ne...  
4                  fair enough kindle2 think perfect :)  


In [40]:
tfidf_vectors_test=tfidfVectorizer.transform(tweets_test['tweet'].tolist())
print(tfidf_vectors_test.shape)

(359, 325212)


In [41]:
pred_y = loaded_model.predict(tfidf_vectors_test.todense())

In [57]:
find_accuracy(list(tweets_test.index),pred_y)

0.4986072423398329